In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os, json

In [3]:
# Check GPU availability
print(tf.test.is_gpu_available())
print(tf.test.is_built_with_cuda())

True
True


In [4]:
DATAPATH = 'data'

In [5]:
# load metadata
csv_filepath ="/".join([DATAPATH, 'metadata.csv'])
df = pd.read_csv(csv_filepath)

In [6]:
df.head()

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
0,vho70jcx,f056da9c64fbf00a4645ae326e8a4339d015d155,biorxiv,SIANN: Strain Identification by Alignment to N...,10.1101/001727,NaN,NaN,biorxiv,Next-generation sequencing is increasingly bei...,2014-01-10,Samuel Minot; Stephen D Turner; Krista L Ternu...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/001727
1,i9tbix2v,daf32e013d325a6feb80e83d15aabc64a48fae33,biorxiv,Spatial epidemiology of networked metapopulati...,10.1101/003889,NaN,NaN,biorxiv,An emerging disease is one infectious epidemic...,2014-06-04,Lin WANG; Xiang Li,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/003889
2,62gfisc6,f33c6d94b0efaa198f8f3f20e644625fa3fe10d2,biorxiv,Sequencing of the human IG light chain loci fr...,10.1101/006866,NaN,NaN,biorxiv,Germline variation at immunoglobulin gene (IG)...,2014-07-03,Corey T Watson; Karyn Meltz Steinberg; Tina A ...,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/006866
3,058r9486,4da8a87e614373d56070ed272487451266dce919,biorxiv,Bayesian mixture analysis for metagenomic comm...,10.1101/007476,NaN,NaN,biorxiv,Deep sequencing of clinical samples is now an ...,2014-07-25,Sofia Morfopoulou; Vincent Plagnol,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/007476
4,wich35l7,eccef80cfbe078235df22398f195d5db462d8000,biorxiv,Mapping a viral phylogeny onto outbreak trees ...,10.1101/010389,NaN,NaN,biorxiv,Developing methods to reconstruct transmission...,2014-11-11,Stephen P Velsko; Jonathan E Allen,NaN,NaN,NaN,True,biorxiv_medrxiv,https://doi.org/10.1101/010389


In [7]:
# Check shape
df.shape

(45774, 17)

In [48]:
import os, os.path

DIR = 'data'
subdirectory = ['biorxiv_medrxiv', 'comm_use_subset', 'custom_license', 'noncomm_use_subset']

total_files = 0
for subdir in subdirectory:
    path = DIR + '/' + subdir + '/' + subdir
    print(subdir, len(os.listdir(path)))
    total_files += len(os.listdir(path))
total_files

biorxiv_medrxiv 1053
comm_use_subset 9315
custom_license 20657
noncomm_use_subset 2350


33375

In [56]:
print(df['full_text_file'].unique())
print(df['full_text_file'].shape)

['biorxiv_medrxiv' 'noncomm_use_subset' 'comm_use_subset' nan
 'custom_license']
(45774,)


In [74]:
print(df[df['full_text_file'].notna() == True]['full_text_file'].unique())
print(df[df['full_text_file'].notna() == True]['full_text_file'].shape)
df_sub = df[df['full_text_file'].notna() == True]

['biorxiv_medrxiv' 'noncomm_use_subset' 'comm_use_subset' 'custom_license']
(35558,)


In [75]:
total_files = 0
for subdir in subdirectory:
    df_temp = df[df['full_text_file'] == subdir]
    print(subdir, len(df_temp))
    total_files += len(df_temp)
total_files

biorxiv_medrxiv 1218
comm_use_subset 8982
custom_license 23227
noncomm_use_subset 2131


35558

In [79]:
# I checked, there are no duplicates is the full_text_file directories
total_files = []
for subdir in subdirectory:
    path = DIR + '/' + subdir + '/' + subdir
    print(subdir, len(os.listdir(path)))
    dir_set = set(os.listdir(path))
    print(subdir, len(dir_set))
    
    lst = [os.path.splitext(x)[0] for x in os.listdir(path)]
    total_files.extend(lst)
len(total_files)

biorxiv_medrxiv 1053
biorxiv_medrxiv 1053
comm_use_subset 9315
comm_use_subset 9315
custom_license 20657
custom_license 20657
noncomm_use_subset 2350
noncomm_use_subset 2350


33375

In [82]:
df_sub = df_sub[df_sub['sha'].isin(total_files)]
df_sub.shape

(30206, 17)

In [86]:
df_sub[df_sub['has_full_text'].notna() == True].shape

(30206, 17)

In [89]:
# These titles don't exist. These files don't exist. 
df_sub[df_sub['sha'].duplicated()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
31108,h46p2oj4,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1016/s0012-3692(15)31462-8,NaN,NaN,els-covid,NaN,2004-12-31,"Filice, Gregory A.",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0012-3692(15)31462-8
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198.0,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6
33298,l2banbg8,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,A pamphlet to answer patients' questions,10.1016/s0140-6736(75)80060-2,NaN,78196.0,els-covid,NaN,1975-10-25,"Steele, S.J.; Goodwin, M.F.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80060-2
33325,ouie3dp8,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,CORONAVIRUS-LIKE PARTICLES IN DIARRHOEA STOOLS,10.1016/s0140-6736(80)92433-2,NaN,6109182.0,els-covid,NaN,1980-12-27,"Curry, A.; Paver, W.K.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92433-2
41028,bcn6tpnx,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-34 A fast procedure for the detection of t...,10.1016/s1386-6532(09)70130-1,NaN,NaN,els-covid,NaN,2009-09-30,"Allice, V.; Milia, M.G.; Gregori, G.; Proia, M...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70130-1
41029,n8n1folf,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-35 Evaluation of two newly developed QIAsy...,10.1016/s1386-6532(09)70131-3,NaN,NaN,els-covid,NaN,2009-09-30,"Hanselle, T.; Kasper, Y.; Otte, M.; Ritter, S....",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70131-3
41030,jof3mnnk,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-36 Performance of the Qiagen Resplex II ve...,10.1016/s1386-6532(09)70132-5,NaN,NaN,els-covid,NaN,2009-09-30,"Halligan, E.; Bible, J.; Sehmi, J.; Dunn, H.; ...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70132-5
44847,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patien...,10.3346/jkms.2020.35.e88,PMC7036343,32080992.0,cc-by-nc,NaN,2020-02-20,"Kim, Jin Yong",J Korean Med Sci,NaN,#1608,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
31108,h46p2oj4,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1016/s0012-3692(15)31462-8,NaN,NaN,els-covid,NaN,2004-12-31,"Filice, Gregory A.",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0012-3692(15)31462-8
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198.0,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6
33298,l2banbg8,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,A pamphlet to answer patients' questions,10.1016/s0140-6736(75)80060-2,NaN,78196.0,els-covid,NaN,1975-10-25,"Steele, S.J.; Goodwin, M.F.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80060-2
33325,ouie3dp8,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,CORONAVIRUS-LIKE PARTICLES IN DIARRHOEA STOOLS,10.1016/s0140-6736(80)92433-2,NaN,6109182.0,els-covid,NaN,1980-12-27,"Curry, A.; Paver, W.K.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92433-2
41028,bcn6tpnx,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-34 A fast procedure for the detection of t...,10.1016/s1386-6532(09)70130-1,NaN,NaN,els-covid,NaN,2009-09-30,"Allice, V.; Milia, M.G.; Gregori, G.; Proia, M...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70130-1
41029,n8n1folf,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-35 Evaluation of two newly developed QIAsy...,10.1016/s1386-6532(09)70131-3,NaN,NaN,els-covid,NaN,2009-09-30,"Hanselle, T.; Kasper, Y.; Otte, M.; Ritter, S....",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70131-3
41030,jof3mnnk,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-36 Performance of the Qiagen Resplex II ve...,10.1016/s1386-6532(09)70132-5,NaN,NaN,els-covid,NaN,2009-09-30,"Halligan, E.; Bible, J.; Sehmi, J.; Dunn, H.; ...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70132-5
44847,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patien...,10.3346/jkms.2020.35.e88,PMC7036343,32080992.0,cc-by-nc,NaN,2020-02-20,"Kim, Jin Yong",J Korean Med Sci,NaN,#1608,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...


In [9]:
df_ft = df[df['has_full_text'] == True]
df_ft.shape

(31753, 17)

In [10]:
df_ft[df_ft['sha'].notna()==False].shape

(0, 17)

In [34]:
df_ft['sha'][df_ft['sha'].duplicated()]

27528    45e40b2d7d973ed5c9798da613fb3cfa4427e2e2
31108    f3aafdecdc43a3f57e58cf6dcea038b1834a953e
33297    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33298    9ce0a6cfd53840cd985f7a1439708c7a48bb7f23
33325    ba4afe00e152de82121a4445aed52c46833d927e
41028    4644c32551fb23aa873a7738ecc8d777bd49877e
41029    4644c32551fb23aa873a7738ecc8d777bd49877e
41030    4644c32551fb23aa873a7738ecc8d777bd49877e
44847    58be092086c74c58e9067121a6ba4836468e7ec3
Name: sha, dtype: object

In [84]:
df_sub[df_sub['sha'].duplicated()]

,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,Microsoft Academic Paper ID,WHO #Covidence,has_full_text,full_text_file,url
27528,c75e60nm,45e40b2d7d973ed5c9798da613fb3cfa4427e2e2,Elsevier,Books received,10.1016/0167-7799(89)90109-1,NaN,NaN,els-covid,NaN,1989-08-31,NaN,Trends in Biotechnology,NaN,NaN,True,custom_license,https://doi.org/10.1016/0167-7799(89)90109-1
31108,h46p2oj4,f3aafdecdc43a3f57e58cf6dcea038b1834a953e,Elsevier,Primum Non Nocere,10.1016/s0012-3692(15)31462-8,NaN,NaN,els-covid,NaN,2004-12-31,"Filice, Gregory A.",Chest,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0012-3692(15)31462-8
33297,fa21ckbr,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,Hypo-osmolality in beer drinkers,10.1016/s0140-6736(75)80059-6,NaN,78198.0,els-covid,NaN,1975-10-25,"Hilden, Tage; Svendsen, Tage Lysbo",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80059-6
33298,l2banbg8,9ce0a6cfd53840cd985f7a1439708c7a48bb7f23,Elsevier,A pamphlet to answer patients' questions,10.1016/s0140-6736(75)80060-2,NaN,78196.0,els-covid,NaN,1975-10-25,"Steele, S.J.; Goodwin, M.F.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(75)80060-2
33325,ouie3dp8,ba4afe00e152de82121a4445aed52c46833d927e,Elsevier,CORONAVIRUS-LIKE PARTICLES IN DIARRHOEA STOOLS,10.1016/s0140-6736(80)92433-2,NaN,6109182.0,els-covid,NaN,1980-12-27,"Curry, A.; Paver, W.K.",The Lancet,NaN,NaN,True,custom_license,https://doi.org/10.1016/s0140-6736(80)92433-2
41028,bcn6tpnx,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-34 A fast procedure for the detection of t...,10.1016/s1386-6532(09)70130-1,NaN,NaN,els-covid,NaN,2009-09-30,"Allice, V.; Milia, M.G.; Gregori, G.; Proia, M...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70130-1
41029,n8n1folf,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-35 Evaluation of two newly developed QIAsy...,10.1016/s1386-6532(09)70131-3,NaN,NaN,els-covid,NaN,2009-09-30,"Hanselle, T.; Kasper, Y.; Otte, M.; Ritter, S....",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70131-3
41030,jof3mnnk,4644c32551fb23aa873a7738ecc8d777bd49877e,Elsevier,PIV-36 Performance of the Qiagen Resplex II ve...,10.1016/s1386-6532(09)70132-5,NaN,NaN,els-covid,NaN,2009-09-30,"Halligan, E.; Bible, J.; Sehmi, J.; Dunn, H.; ...",Journal of Clinical Virology,NaN,NaN,True,custom_license,https://doi.org/10.1016/s1386-6532(09)70132-5
44847,u4taazr3,58be092086c74c58e9067121a6ba4836468e7ec3,PMC,Letter to the Editor: Case of the Index Patien...,10.3346/jkms.2020.35.e88,PMC7036343,32080992.0,cc-by-nc,NaN,2020-02-20,"Kim, Jin Yong",J Korean Med Sci,NaN,#1608,True,noncomm_use_subset,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7...
